In [1]:
pip install rasterio cupy-cuda11x

   ---------------------------------------- 0.0/76.9 MB ? eta -:--:--
    --------------------------------------- 1.3/76.9 MB 7.0 MB/s eta 0:00:11
   - -------------------------------------- 2.6/76.9 MB 6.9 MB/s eta 0:00:11
   -- ------------------------------------- 4.2/76.9 MB 6.9 MB/s eta 0:00:11
   -- ------------------------------------- 5.8/76.9 MB 7.1 MB/s eta 0:00:11
   --- ------------------------------------ 7.3/76.9 MB 7.1 MB/s eta 0:00:10
   ---- ----------------------------------- 8.9/76.9 MB 7.2 MB/s eta 0:00:10
   ----- ---------------------------------- 10.5/76.9 MB 7.3 MB/s eta 0:00:10
   ------ --------------------------------- 12.6/76.9 MB 7.7 MB/s eta 0:00:09
   ------- -------------------------------- 14.7/76.9 MB 8.1 MB/s eta 0:00:08
   -------- ------------------------------- 17.3/76.9 MB 8.5 MB/s eta 0:00:08
   ---------- ----------------------------- 19.7/76.9 MB 8.8 MB/s eta 0:00:07
   ---------- ----------------------------- 20.2/76.9 MB 8.2 MB/s eta 0:00:07


In [2]:
#SolarArea 생성
import rasterio
import rasterio.mask
import numpy as np
import os
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Hillshade 폴더 및 Masking 파일 경로
hillshade_folder = r"E:\2025_Jeju_SectorCoupling\Hillshade"
mask_file = r"C:\Users\82105\Downloads\masked_output.tif"
output_folder = r"E:\2025_Jeju_SectorCoupling\Masked_Hill"

# Hillshade 폴더 내 모든 raster 파일 찾기
hillshade_files = [os.path.join(hillshade_folder, f) for f in os.listdir(hillshade_folder) if f.endswith(".tif")]

# Mask 파일 로드
with rasterio.open(mask_file) as mask_src:
    mask_data = mask_src.read(1)  # 첫 번째 밴드 사용
    mask_transform = mask_src.transform
    mask_crs = mask_src.crs
    mask_res = mask_src.res  # 셀 크기 (해상도)

# Hillshade 파일 처리
for hillshade_path in hillshade_files:
    with rasterio.open(hillshade_path) as hillshade_src:
        hillshade_data = hillshade_src.read(1)
        hillshade_transform = hillshade_src.transform
        hillshade_crs = hillshade_src.crs
        hillshade_res = hillshade_src.res  # 셀 크기 (해상도)

        # 좌표계 확인 및 변환
        if hillshade_crs != mask_crs:
            raise ValueError(f"{hillshade_path}의 좌표계가 mask 파일과 다릅니다. 변환이 필요합니다.")

        # 셀 크기 비교 후 리샘플링 수행
        if hillshade_res != mask_res:
            dst_transform, width, height = calculate_default_transform(
                hillshade_crs, mask_crs, mask_src.width, mask_src.height, *mask_src.bounds
            )
            resampled_hillshade = np.empty((height, width), dtype=hillshade_data.dtype)

            reproject(
                source=hillshade_data,
                destination=resampled_hillshade,
                src_transform=hillshade_transform,
                src_crs=hillshade_crs,
                dst_transform=dst_transform,
                dst_crs=mask_crs,
                resampling=Resampling.bilinear
            )
            hillshade_data = resampled_hillshade
            hillshade_transform = dst_transform

        # **Masking 수행: mask 값이 0 또는 null이면 0, 1이면 Hillshade 유지**
        masked_hillshade = np.where((mask_data == 1), hillshade_data, 0)

        # 출력 파일 저장
        output_path = os.path.join(output_folder, os.path.basename(hillshade_path))
        out_meta = hillshade_src.meta.copy()
        out_meta.update({"transform": hillshade_transform, "width": mask_src.width, "height": mask_src.height})

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(masked_hillshade, 1)

        print(f"처리 완료: {output_path}")

처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 08-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 09-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 10-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 11-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 12-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 13-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 14-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 15-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 16-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-01 17-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_Hill\hillshade_2024-01-02 08-00-00+09-00.tif
처리 완료: E:\2025_Jeju_SectorCoupling\Masked_H

In [4]:
import cupy
print(cupy.__version__)



13.4.1


In [5]:
import cupy as cp
print(cp.get_device_count())  # 사용 가능한 GPU 디바이스 수 확인


AttributeError: module 'cupy' has no attribute 'get_device_count'

In [7]:
#제주,고산,서귀포_PVgen데이터합성
import os
import re
import numpy as np
import rasterio
from rasterio.transform import from_origin
import xml.etree.ElementTree as ET

# 입력 및 출력 디렉토리 설정
dirs = {
    "jeju": r"E:\2025_Jeju_SectorCoupling\PVgen_Jeju",
    "gosan": r"E:\2025_Jeju_SectorCoupling\PVgen_Gosan",
    "seoguipo": r"E:\2025_Jeju_SectorCoupling\PVgen_Seoguipo",
}
output_dir = r"E:\2025_Jeju_SectorCoupling\PVgen"
os.makedirs(output_dir, exist_ok=True)

# 래스터 파일 정렬
files = {k: sorted([f for f in os.listdir(v) if f.endswith(".tif")]) for k, v in dirs.items()}

# 파일 개수 검증
if len(set(map(len, files.values()))) != 1:
    raise ValueError("각 디렉토리에 동일한 개수의 래스터 파일이 있어야 합니다.")

# 파일명 정리 함수
def remove_prefix_before_date(filename):
    return re.sub(r"^.*?(\d{4}-\d{2}-\d{2}.*)", r"\1", filename)

# TFW 파일 생성 함수
def save_tfw(transform, tfw_path):
    """World 파일(.tfw) 저장"""
    with open(tfw_path, 'w') as f:
        f.write(f"{transform.a:.10f}\n")  # X pixel size
        f.write(f"{transform.b:.10f}\n")  # Rotation (usually 0)
        f.write(f"{transform.d:.10f}\n")  # Rotation (usually 0)
        f.write(f"{transform.e:.10f}\n")  # Y pixel size (negative)
        f.write(f"{transform.xoff:.10f}\n")  # X coordinate of top left pixel
        f.write(f"{transform.yoff:.10f}\n")  # Y coordinate of top left pixel

# XML 메타데이터 저장 함수
def save_xml(metadata, xml_path):
    """XML 메타데이터 저장"""
    root = ET.Element("RasterMetadata")
    ET.SubElement(root, "CRS").text = metadata["crs"]
    ET.SubElement(root, "NoDataValue").text = str(metadata["nodata_value"])
    
    transform = ET.SubElement(root, "Transform")
    ET.SubElement(transform, "PixelSizeX").text = str(metadata["transform"][0])
    ET.SubElement(transform, "PixelSizeY").text = str(metadata["transform"][4])
    ET.SubElement(transform, "UpperLeftX").text = str(metadata["transform"][2])
    ET.SubElement(transform, "UpperLeftY").text = str(metadata["transform"][5])
    
    source_files = ET.SubElement(root, "SourceFiles")
    for src in metadata["source_files"]:
        ET.SubElement(source_files, "File").text = src

    tree = ET.ElementTree(root)
    tree.write(xml_path, encoding="utf-8", xml_declaration=True)

# 래스터 병합 및 저장
for f1, f2, f3 in zip(files["jeju"], files["gosan"], files["seoguipo"]):
    raster_paths = {k: os.path.join(dirs[k], f) for k, f in zip(dirs.keys(), [f1, f2, f3])}
    
    # TIFF 파일 불러오기 및 NumPy 배열 변환
    rasters = {k: rasterio.open(path) for k, path in raster_paths.items()}
    arrays = {k: rasters[k].read(1).astype(np.float32) for k in rasters}

    # NoData(Null) 값을 0으로 변환 후 합산
    result_arr = sum(np.nan_to_num(arr, nan=0) for arr in arrays.values())

    # 좌표 변환 정보 유지
    ref_raster = rasters["jeju"]
    transform = ref_raster.transform
    profile = ref_raster.profile.copy()
    
    # 출력 파일 경로 설정
    output_filename = f"TotalSolargen_{remove_prefix_before_date(f1)}"
    output_tif = os.path.join(output_dir, output_filename + ".tif")
    output_tfw = os.path.join(output_dir, output_filename + ".tfw")
    output_xml = os.path.join(output_dir, output_filename + ".xml")
    
    # TIFF 저장
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(result_arr, 1)

    # TFW 파일 저장
    save_tfw(transform, output_tfw)

    # XML 메타데이터 저장
    metadata = {
        "source_files": [f1, f2, f3],
        "transform": transform.to_gdal(),  # GDAL 형식 변환
        "crs": str(ref_raster.crs),
        "nodata_value": profile.get("nodata", None)
    }
    save_xml(metadata, output_xml)
    
    print(f"결과 저장: {output_tif}")
    print(f"TFW 저장: {output_tfw}")
    print(f"XML 메타데이터 저장: {output_xml}")

print("모든 래스터 병합이 완료되었습니다.")

os.makedirs(output_dir, exist_ok=True)

# 래스터 파일 정렬
files = {k: sorted(f for f in os.listdir(v) if f.endswith(".tif")) for k, v in dirs.items()}
if len(set(map(len, files.values()))) != 1:
    raise ValueError("각 디렉토리에 동일한 개수의 래스터 파일이 있어야 합니다.")

# 파일명 정리 함수
def clean_filename(filename):
    return re.sub(r"^.*?(\d{4}-\d{2}-\d{2}.*)", r"\1", filename)

# TFW 및 XML 저장 함수
def save_tfw(transform, path):
    with open(path, 'w') as f:
        f.writelines(f"{v:.10f}\n" for v in [transform.a, transform.b, transform.d, transform.e, transform.xoff, transform.yoff])

def save_xml(metadata, path):
    root = ET.Element("RasterMetadata")
    ET.SubElement(root, "CRS").text = metadata["crs"]
    ET.SubElement(root, "NoDataValue").text = str(metadata["nodata"])
    transform = ET.SubElement(root, "Transform")
    for k, v in zip(["PixelSizeX", "PixelSizeY", "UpperLeftX", "UpperLeftY"], metadata["transform"][:6:2]):
        ET.SubElement(transform, k).text = str(v)
    sources = ET.SubElement(root, "SourceFiles")
    [ET.SubElement(sources, "File").text for f in metadata["source_files"]]
    ET.ElementTree(root).write(path, encoding="utf-8", xml_declaration=True)

# 래스터 병합 및 저장
for f1, f2, f3 in zip(*files.values()):
    raster_paths = {k: os.path.join(dirs[k], f) for k, f in zip(dirs, [f1, f2, f3])}
    with rasterio.open(raster_paths["jeju"]) as ref:
        transform, profile = ref.transform, ref.profile.copy()
        arrays = [np.nan_to_num(rasterio.open(r).read(1), nan=0) for r in raster_paths.values()]
        result_arr = sum(arrays)
    
    output_name = f"TotalSolargen_{clean_filename(f1)}"
    output_paths = {ext: os.path.join(output_dir, output_name + ext) for ext in [".tif", ".tfw", ".xml"]}
    
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_paths[".tif"], 'w', **profile) as dst:
        dst.write(result_arr, 1)
    
    save_tfw(transform, output_paths[".tfw"])
    save_xml({"source_files": [f1, f2, f3], "transform": transform.to_gdal(), "crs": str(ref.crs), "nodata": profile.get("nodata", None)}, output_paths[".xml"])
    
    print(f"저장 완료: {output_paths}")

print("모든 래스터 병합 완료.")



결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 13-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 13-00-00+09-00.tif.tfw
XML 메타데이터 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 13-00-00+09-00.tif.xml
결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 14-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 14-00-00+09-00.tif.tfw
XML 메타데이터 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 14-00-00+09-00.tif.xml
결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 15-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 15-00-00+09-00.tif.tfw
XML 메타데이터 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 15-00-00+09-00.tif.xml
결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 16-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 16-00-00+09-00.tif.

KeyboardInterrupt: 

In [8]:
pip install geopandas shapely fiona oneapi


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement oneapi (from versions: none)
ERROR: No matching distribution found for oneapi


In [1]:
pip install rasterstats

Note: you may need to restart the kernel to use updated packages.


In [14]:
import geopandas as gpd
from shapely.geometry import Polygon
import warnings

# 경고 무시 설정 (유니코드 오류로 인한 경고)
warnings.filterwarnings("ignore", category=RuntimeWarning)

def read_shapefile_with_encoding(input_shapefile):
    # 시도할 인코딩 리스트
    encodings = ['euc-kr', 'cp949', 'ISO-8859-1', 'utf-8']
    
    # 인코딩 순차적으로 시도
    for encoding in encodings:
        try:
            print(f"시도 중: {encoding}")
            gdf = gpd.read_file(input_shapefile, encoding=encoding)
            return gdf
        except UnicodeDecodeError as e:
            print(f"인코딩 오류 발생 ({encoding}): {e}")
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}")
    
    # 만약 모든 인코딩이 실패할 경우
    raise ValueError("모든 인코딩 시도에서 실패했습니다. 파일을 확인해주세요.")

# 입력 파일 및 출력 파일 경로
input_shapefile = r"E:\2025_Jeju_SectorCoupling\Jeju_resi.shp"
output_shapefile = r"E:\2025_Jeju_SectorCoupling\Jeju_Masking_buffer.shp"
buffer_distance = 100  # 100 미터

# 인코딩 오류 처리 후 Shapefile 읽기
try:
    gdf = read_shapefile_with_encoding(input_shapefile)
    
    # 좌표계가 미터로 설정되어 있는지 확인하고, 아니면 변환
    if gdf.crs != 'EPSG:3395':  # EPSG:3395는 미터 단위의 CRS
        gdf = gdf.to_crs(epsg=3395)

    # 버퍼 생성 (단위는 미터)
    gdf['buffer'] = gdf.geometry.buffer(buffer_distance)

    # 결과를 새로운 Shapefile로 저장
    gdf[['buffer']].to_file(output_shapefile)

    print(f"버퍼가 {output_shapefile}에 생성되었습니다.")

except ValueError as e:
    print(f"Shapefile을 읽을 수 없습니다: {e}")



시도 중: euc-kr
버퍼가 E:\2025_Jeju_SectorCoupling\Jeju_Masking_buffer.shp에 생성되었습니다.


In [15]:
#merge
import geopandas as gpd
import pandas as pd
import warnings

# 경고 무시 설정 (유니코드 오류로 인한 경고)
warnings.filterwarnings("ignore", category=RuntimeWarning)

def read_shapefile_with_encoding(input_shapefile):
    # 시도할 인코딩 리스트
    encodings = ['euc-kr', 'cp949', 'ISO-8859-1', 'utf-8']
    
    # 인코딩 순차적으로 시도
    for encoding in encodings:
        try:
            print(f"시도 중: {encoding}")
            gdf = gpd.read_file(input_shapefile, encoding=encoding)
            return gdf
        except UnicodeDecodeError as e:
            print(f"인코딩 오류 발생 ({encoding}): {e}")
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}")
    
    # 만약 모든 인코딩이 실패할 경우
    raise ValueError("모든 인코딩 시도에서 실패했습니다. 파일을 확인해주세요.")

# 입력 파일 경로 설정 (3개의 Shapefile)
input_shapefile_1 = r"E:\2025_Jeju_SectorCoupling\JejuNationalPark.shp"
input_shapefile_2 = r"E:\2025_Jeju_SectorCoupling\Jeju_Masking_buffer.shp"
input_shapefile_3 = r"E:\2025_Jeju_SectorCoupling\TL_SPRD_RW_50000.shp"
# 출력 파일 경로 설정
output_shapefile = r"E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp"

# Shapefile 읽기
gdf1 = read_shapefile_with_encoding(input_shapefile_1)
gdf2 = read_shapefile_with_encoding(input_shapefile_2)
gdf3 = read_shapefile_with_encoding(input_shapefile_3)

# 좌표계가 일치하는지 확인하고, 일치하지 않으면 변환
if gdf1.crs != gdf2.crs:
    gdf2 = gdf2.to_crs(gdf1.crs)
if gdf1.crs != gdf3.crs:
    gdf3 = gdf3.to_crs(gdf1.crs)

# GeoDataFrame 병합 (pd.concat 사용)
gdf_merged = pd.concat([gdf1, gdf2, gdf3], ignore_index=True)

# 병합된 데이터를 새로운 Shapefile로 저장
gdf_merged.to_file(output_shapefile)

print(f"병합된 Shapefile이 {output_shapefile}에 생성되었습니다.")

시도 중: euc-kr
시도 중: euc-kr
시도 중: euc-kr
병합된 Shapefile이 E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp에 생성되었습니다.


In [18]:
import geopandas as gpd

# "Jeju_Masked.shp" 파일 경로
masked_shp_file = r"E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp"

# GeoPandas로 shp 파일 읽기
gdf = gpd.read_file(masked_shp_file)

# 'Mask' 필드의 모든 값을 0으로 설정
gdf['Mask'] = 0

# 수정된 GeoDataFrame을 다시 shapefile로 저장
gdf.to_file(masked_shp_file)

# 저장 완료 메시지 출력
print(f"{masked_shp_file}의 'Mask' 필드 값을 모두 0으로 설정했습니다.")


E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp의 'Mask' 필드 값을 모두 0으로 설정했습니다.


In [12]:
#Pvgen shp 배정
import os
import re
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping

# 특수문자 제거 함수
def clean_filename(filename):
    return re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)

# 입력 폴더 경로 및 출력 폴더 경로 설정
input_raster_folder = r"E:\2025_Jeju_SectorCoupling\PVgen"
shp_file = r"E:\2025_Jeju_SectorCoupling\LSMD_ADM_SECT_UMD_50_202503.shp"
output_folder = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon"

# 출력 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# GeoPandas로 shp 파일 읽기
gdf = gpd.read_file(shp_file)
gdf['pvgen'] = 0.0  # 'pvgen' 필드 추가

# 이전 파일의 CRS 저장 변수
previous_crs = None

# tif 파일 리스트 정렬
tif_files = sorted(f for f in os.listdir(input_raster_folder) if f.endswith(".tif"))

# tif 파일 순회
for raster_file in tif_files:
    raster_path = os.path.join(input_raster_folder, raster_file)
    cleaned_raster_file = clean_filename(raster_file)

    # 래스터 파일 읽기
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs  # 좌표계 가져오기

        # CRS가 없으면 이전 파일의 CRS 사용
        if raster_crs is None:
            if previous_crs is not None:
                raster_crs = previous_crs
            else:
                print(f"Error: {raster_file} has no CRS and no previous CRS available. Skipping file.")
                continue

        previous_crs = raster_crs

        # 좌표계가 다르면 변환
        if gdf.crs != raster_crs:
            gdf = gdf.to_crs(raster_crs)

        # 폴리곤에 해당하는 마스크 처리 및 pvgen 계산
        for idx, row in gdf.iterrows():
            polygon = row['geometry']
            geo = [mapping(polygon)]

            # 마스크 적용
            try:
                out_image, _ = mask(src, geo, crop=True)
                if src.nodata is not None:
                    out_image = np.ma.masked_equal(out_image, src.nodata)

                # pvgen 값 계산 후 업데이트
                total_sum = np.sum(out_image) if np.sum(out_image) != 0 else 0
                gdf.at[idx, 'pvgen'] = total_sum / 3600

            except Exception as e:
                print(f"Error processing polygon {idx} in {raster_file}: {e}")

    # 결과를 새로운 shp 파일로 저장
    output_shp = os.path.join(output_folder, f"{os.path.splitext(cleaned_raster_file)[0]}_Polygon.shp")
    gdf_copy = gdf.copy()  # 기존 gdf 복사

    # 새 shp 파일로 저장
    gdf_copy.to_file(output_shp)
    print(f"{output_shp} 저장 완료했습니다.")

# 종료 메시지
print("모든 래스터 파일에 대해 연산을 완료했습니다.")

E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_08-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_09-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_10-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_11-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_12-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_13-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_14-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_15-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_16-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen

KeyboardInterrupt: 

In [22]:
#Pvgen shp 배정
import os
import re
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping

# 특수문자 제거 함수
def clean_filename(filename):
    return re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)

# 입력 폴더 경로 및 출력 폴더 경로 설정
input_raster_folder = r"E:\2025_Jeju_SectorCoupling\PVgen"
shp_file = r"E:\2025_Jeju_SectorCoupling\LSMD_ADM_SECT_UMD_50_202503.shp"
output_folder = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon"

# 출력 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# GeoPandas로 shp 파일 읽기
gdf = gpd.read_file(shp_file)
gdf['pvgen'] = 0.0  # 'pvgen' 필드 추가

# 이전 파일의 CRS 저장 변수
previous_crs = None

# tif 파일 리스트 정렬
tif_files = sorted(f for f in os.listdir(input_raster_folder) if f.endswith(".tif"))

# tif 파일 순회
for raster_file in tif_files:
    raster_path = os.path.join(input_raster_folder, raster_file)
    cleaned_raster_file = clean_filename(raster_file)

    # 래스터 파일 읽기
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs  # 좌표계 가져오기

        # CRS가 없으면 이전 파일의 CRS 사용
        if raster_crs is None:
            if previous_crs is not None:
                raster_crs = previous_crs
            else:
                print(f"Error: {raster_file} has no CRS and no previous CRS available. Skipping file.")
                continue

        previous_crs = raster_crs

        # 좌표계가 다르면 변환
        if gdf.crs != raster_crs:
            gdf = gdf.to_crs(raster_crs)

        # 폴리곤에 해당하는 마스크 처리 및 pvgen 계산
        for idx, row in gdf.iterrows():
            polygon = row['geometry']
            geo = [mapping(polygon)]

            # 마스크 적용
            try:
                out_image, _ = mask(src, geo, crop=True)
                if src.nodata is not None:
                    out_image = np.ma.masked_equal(out_image, src.nodata)

                # pvgen 값 계산 후 업데이트
                total_sum = np.sum(out_image) if np.sum(out_image) != 0 else 0
                gdf.at[idx, 'pvgen'] = total_sum / 3600

            except Exception as e:
                print(f"Error processing polygon {idx} in {raster_file}: {e}")

    # 결과를 새로운 shp 파일로 저장
    output_shp = os.path.join(output_folder, f"{os.path.splitext(cleaned_raster_file)[0]}_Polygon.shp")
    gdf_copy = gdf.copy()  # 기존 gdf 복사

    # 새 shp 파일로 저장
    gdf_copy.to_file(output_shp)
    print(f"{output_shp} 저장 완료했습니다.")

    # 새로 생성된 shp 파일에서 필드 확인
    try:
        new_gdf = gpd.read_file(output_shp)

        # 'EMD_CD'와 'pvgen' 필드가 있는지 확인
        if 'EMD_CD' in new_gdf.columns and 'pvgen' in new_gdf.columns:
            # 필드가 존재하면 값이 있는지 확인
            if new_gdf['pvgen'].isnull().any() or new_gdf['EMD_CD'].isnull().any():
                print(f"Error: Missing values in 'EMD_CD' or 'pvgen' in {output_shp}. Retrying...")
                # pvgen 계산을 다시 실행하도록 gdf를 업데이트하고 재처리
                gdf = gpd.read_file(shp_file)  # 원본 shp 파일을 다시 읽어서 초기화
                gdf['pvgen'] = 0.0  # 'pvgen' 필드 초기화
                continue  # 파일을 다시 처리

        else:
            print(f"Error: 'EMD_CD' or 'pvgen' field is missing in {output_shp}. Retrying...")
            # 필드가 없으면 다시 처리
            gdf = gpd.read_file(shp_file)  # 원본 shp 파일을 다시 읽어서 초기화
            gdf['pvgen'] = 0.0  # 'pvgen' 필드 초기화
            continue  # 파일을 다시 처리

    except Exception as e:
        print(f"Error reading the shapefile {output_shp}: {e}")
        continue  # 오류가 발생하면 해당 파일은 건너뛰고 계속 진행

# 종료 메시지
print("모든 래스터 파일에 대해 연산을 완료했습니다.")


E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_08-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_09-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_10-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_11-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_12-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_13-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_14-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_15-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_16-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen

In [ ]:
#shp_id 확인인
import geopandas as gpd
import pandas as pd
import os
import sys

# 입력 폴더 경로
input_folder = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon"
csv_path = r"E:\2025_Jeju_SectorCoupling\2024Jeju_1hour_new.csv"

# 폴더 내 모든 .shp 파일 가져오기
shp_files = [f for f in os.listdir(input_folder) if f.endswith(".shp")]

if not shp_files:
    print("⚠️ 처리할 .shp 파일이 없습니다.")
    sys.exit()

# CSV 데이터 불러오기 (인코딩 오류 방지)
encoding_list = ['utf-8', 'cp949', 'euc-kr']
for enc in encoding_list:
    try:
        df = pd.read_csv(csv_path, encoding=enc, header=None)
        break
    except UnicodeDecodeError:
        continue
else:
    raise ValueError("CSV 파일을 읽을 수 없습니다. 지원되지 않는 인코딩 형식입니다.")

# CSV 1행 1열부터 74열까지 ID 데이터로 사용
ids = df.iloc[0, :74].astype(float).astype(int).astype(str).tolist()
print(ids)

['50130116', '50130253', '50130250', '50130114', '50130117', '50130105', '50130102', '50130110', '50130120', '50130107', '50130119', '50130101', '50130103', '50130106', '50130259', '50130109', '50130310', '50130115', '50130112', '50130111', '50130320', '50130121', '50130118', '50130108', '50130104', '50130113', '50110107', '50110256', '50110140', '50110122', '50110129', '50110128', '50110127', '50110131', '50110130', '50110138', '50110116', '50110106', '50110105', '50110115', '50110114', '50110113', '50110118', '50110117', '50110253', '50110137', '50110136', '50110134', '50110121', '50110120', '50110119', '50110124', '50110123', '50110132', '50110110', '50110109', '50110108', '50110330', '50110135', '50110104', '50110103', '50110126', '50110125', '50110102', '50110101', '50110259', '50110320', '50110310', '50110250', '50110139', '50110112', '50110111', '50110133', '50130122']


In [23]:
#shp전력소비량배정
import geopandas as gpd
import pandas as pd
import os
import sys

# 입력 폴더 경로
input_folder = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon"
csv_path = r"E:\2025_Jeju_SectorCoupling\2024Jeju_1hour_new.csv"

# 폴더 내 모든 .shp 파일 가져오기
shp_files = [f for f in os.listdir(input_folder) if f.endswith(".shp")]

if not shp_files:
    print("⚠️ 처리할 .shp 파일이 없습니다.")
    sys.exit()

# CSV 데이터 불러오기 (인코딩 오류 방지)
encoding_list = ['utf-8', 'cp949', 'euc-kr']
for enc in encoding_list:
    try:
        df = pd.read_csv(csv_path, encoding=enc, header=None)
        break
    except UnicodeDecodeError:
        continue
else:
    raise ValueError("CSV 파일을 읽을 수 없습니다. 지원되지 않는 인코딩 형식입니다.")

# CSV 1행 1열부터 74열까지 ID 데이터로 사용
ids = df.iloc[0, :74].astype(float).astype(int).astype(str).tolist()
print(ids)
# SHP 파일 처리
for index, filename in enumerate(shp_files):
    shp_path = os.path.join(input_folder, filename)
    gdf = gpd.read_file(shp_path)

    # ElecUse 필드가 없으면 추가
    if "ElecUse" not in gdf.columns:
        gdf["ElecUse"] = 0.0  # 처음에 0.0으로 float 타입으로 초기화

    # ElecUse를 double (float) 타입으로 변환
    gdf["ElecUse"] = gdf["ElecUse"].astype(float)

    for i, shp_id in enumerate(ids):
        # CSV에서 field_value를 double (float)로 처리
        field_value = pd.to_numeric(df.iloc[index + 1, i], errors='coerce')  # CSV n+1행의 값
        
        # NaN 값 처리 (필요시)
        if pd.isna(field_value):
            field_value = 0.0  # 기본값 처리

        # EMD_CD 값 매칭: shp_id와 EMD_CD가 정확히 일치하는지 확인
        mask = gdf["EMD_CD"] == str(shp_id)  # shp_id를 str로 변환 후 비교
        
        # 매칭된 값이 있으면 ElecUse 업데이트
        if mask.any():
            gdf.loc[mask, "ElecUse"] = field_value
        else:
            print(f"⚠️ No match for EMD_CD: {shp_id} in {filename}")

    # 변경된 SHP 파일 저장 (덮어쓰기)
    gdf.to_file(shp_path)
    print(f"✅ {filename} → 'ElecUse' 필드 업데이트 완료")

print("✅ 모든 SHP 파일 업데이트 완료.")




['50130116', '50130253', '50130250', '50130114', '50130117', '50130105', '50130102', '50130110', '50130120', '50130107', '50130119', '50130101', '50130103', '50130106', '50130259', '50130109', '50130310', '50130115', '50130112', '50130111', '50130320', '50130121', '50130118', '50130108', '50130104', '50130113', '50110107', '50110256', '50110140', '50110122', '50110129', '50110128', '50110127', '50110131', '50110130', '50110138', '50110116', '50110106', '50110105', '50110115', '50110114', '50110113', '50110118', '50110117', '50110253', '50110137', '50110136', '50110134', '50110121', '50110120', '50110119', '50110124', '50110123', '50110132', '50110110', '50110109', '50110108', '50110330', '50110135', '50110104', '50110103', '50110126', '50110125', '50110102', '50110101', '50110259', '50110320', '50110310', '50110250', '50110139', '50110112', '50110111', '50110133', '50130122']
✅ TotalSolargen_2024-01-01_08-00-00_09-00_Polygon.shp → 'ElecUse' 필드 업데이트 완료
✅ TotalSolargen_2024-01-01_09-00-0

In [29]:
import geopandas as gpd
import pandas as pd
import os
import sys

# 입력 폴더 경로
input_folder = r"F:\Junkyo\2025 Jeju Sector Coupling\PVgen_polygon70"

# 폴더 내 모든 .shp 파일 가져오기
shp_files = [f for f in os.listdir(input_folder) if f.endswith(".shp")]

if not shp_files:
    print("⚠️ 처리할 .shp 파일이 없습니다.")
    sys.exit()

# Shapefile 처리
for index, filename in enumerate(shp_files):
    shp_path = os.path.join(input_folder, filename)

    # Shapefile 불러오기
    gdf = gpd.read_file(shp_path)

    # Sur_Energy 필드 계산 및 추가
    gdf["Sur_Energy"] = gdf["pvgen"] - 0.7* gdf["ElecUse"]

    # 결과 저장 (덮어쓰기)
    gdf.to_file(shp_path)
    print(f"✅ {filename} → 'Sur_Energy' 필드 추가 완료")

    # 마지막 파일이면 프로그램 종료
    if index == len(shp_files) - 1:
        print("✅ 모든 파일 처리가 완료되었습니다. 프로그램을 종료합니다.")
        sys.exit()




✅ TotalSolargen_2024-01-01_08-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_09-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_10-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_11-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_12-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_13-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_14-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_15-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_16-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-01_17-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-02_08-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-02_09-00-00_09-00_Polygon.shp → 'Sur_Energy' 필드 추가 완료
✅ TotalSolargen_2024-01-02_10-00-00_09-00_Polygon.shp → 'Sur_Ene

SystemExit: 

C:\Users\82105\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
import geopandas as gpd
import pandas as pd
import os

# 파일 경로 설정
csv_path = r"C:\Users\82105\Desktop\Temp.csv"
shp_path = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-12-31_17-00-00_09-00.tif_Polygon.shp"

# CSV 데이터 불러오기 (인코딩 오류 방지)
encoding_list = ['utf-8', 'cp949', 'euc-kr']
for enc in encoding_list:
    try:
        df = pd.read_csv(csv_path, encoding=enc)
        break
    except UnicodeDecodeError:
        continue
else:
    raise ValueError("CSV 파일을 읽을 수 없습니다. 지원되지 않는 인코딩 형식입니다.")

# 4434번째 행 데이터 추출
target_row = df.iloc[0, 0:].tolist()  # 3열부터 데이터 참조
ids = df.iloc[0, 0:].astype(str).tolist()  # 2행을 ID로 사용

# SHP 파일 처리
gdf = gpd.read_file(shp_path)

# "EMD_CD" 필드에 NaN 값이 있을 수 있으므로 처리
gdf["EMD_CD"] = gdf["EMD_CD"].fillna("").astype(str)

# 필드 추가 (이미 존재하면 생략)
if "ElecUse" not in gdf.columns:
    gdf["ElecUse"] = None

# 데이터 매핑
for i, shp_id in enumerate(ids):
    field_value = target_row[i]  # 4434행 데이터 참조
    try:
        gdf.loc[gdf["EMD_CD"].astype(str) == shp_id, "ElecUse"] = field_value
    except KeyError as e:
        print(f"Error with shp_id {shp_id}: {e}")

# 변경된 SHP 파일 저장 (덮어쓰기)
gdf.to_file(shp_path)
print(f"업데이트 완료: {shp_path}")

print("SHP 파일 업데이트 완료")




업데이트 완료: E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-12-31_17-00-00_09-00.tif_Polygon.shp
SHP 파일 업데이트 완료


In [17]:
import geopandas as gpd

# 입력 Shapefile 경로
shp_file = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-12-31_17-00-00_09-00.tif_Polygon.shp"

# Shapefile 불러오기
gdf = gpd.read_file(shp_file)

# 'pvgen' 필드에서 'ElecUse' 필드 값 빼기
gdf["Sur_Energy"] = gdf["pvgen"] - gdf["ElecUse"]

# 결과 저장 (덮어쓰기)
gdf.to_file(shp_file)

print(f"✅ '{shp_file}' → 'Sur_Energy' 필드 추가 완료")


✅ 'E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-12-31_17-00-00_09-00.tif_Polygon.shp' → 'Sur_Energy' 필드 추가 완료


In [30]:
#계절별 평균 계산_봄
import geopandas as gpd
import os
import pandas as pd

# 대상 폴더 경로
folder_path = r"F:\Junkyo\2025 Jeju Sector Coupling\average_spring70"

# 모든 shp 파일 읽고 결합
gdfs = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".shp"):
        file_path = os.path.join(folder_path, file_name)
        gdfs.append(gpd.read_file(file_path))

# 모든 데이터프레임을 하나로 결합 (GeoDataFrame으로 변환)
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), geometry='geometry')

# 데이터 타입 확인 (디버깅용)
print(type(combined_gdf))  # <class 'geopandas.geodataframe.GeoDataFrame'> 이어야 함
print(combined_gdf.columns)  # 컬럼 확인
print(combined_gdf.head())  # 데이터 미리보기

# 동일한 EMD_CD 기준으로 Sur_Energy 평균 계산
if "EMD_CD" in combined_gdf.columns and "Sur_Energy" in combined_gdf.columns:
    averaged_gdf = combined_gdf.groupby("EMD_CD").agg({
        "Sur_Energy": "mean",  # Sur_Energy 평균 계산
        "geometry": "first"    # 첫 번째 geometry 유지
    }).reset_index()

    # 첫 번째 파일의 CRS(좌표 참조 시스템) 유지
    first_file = gdfs[0]
    averaged_gdf = gpd.GeoDataFrame(averaged_gdf, geometry='geometry')  # GeoDataFrame 변환
    averaged_gdf.set_crs(first_file.crs, inplace=True)  # CRS 설정

    # 결과 shp 파일 저장
    output_path = os.path.join(folder_path, "Average_spring.shp")
    averaged_gdf.to_file(output_path)
    print(f"결과가 {output_path}에 저장되었습니다.")
else:
    print("EMD_CD 또는 Sur_Energy 필드가 존재하지 않습니다.")

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['EMD_CD', 'COL_ADM_SE', 'EMD_NM', 'SGG_OID', 'NEAR_FID', 'NEAR_DIST',
       'NEAR_DIST3', 'pvgen', 'ElecUse', 'Sur_Energy', 'geometry'],
      dtype='object')
     EMD_CD COL_ADM_SE   EMD_NM  SGG_OID  NEAR_FID    NEAR_DIST   NEAR_DIST3  \
0  50110131      50110  50110̵3      145         1  5106.865277  5106.865277   
1  50110126      50110    50ȣ1̵      143         1  5900.005278  5900.005278   
2  50110123      50110    5ܵ11ϵ      142         1  8848.691253  8848.691253   
3  50110128      50110  50110̵2      141         1  3661.064454  3661.064454   
4  50110125      50110    50ȣ1ϵ      140         1  6051.042301  6051.042301   

     pvgen    ElecUse  Sur_Energy  \
0  0.06655   8.268767   -5.721587   
1  0.03740  17.240716  -12.031101   
2  0.11990  47.711459  -33.278121   
3  0.08745   3.763297   -2.546858   
4  0.05500  23.929678  -16.695775   

                                            geometry  
0  POLYGON ((126.58464 33.51

In [31]:
#계절별 평균 계산_여름
import geopandas as gpd
import os
import pandas as pd

# 대상 폴더 경로
folder_path = r"F:\Junkyo\2025 Jeju Sector Coupling\average_summer70"

# 모든 shp 파일 읽고 결합
gdfs = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".shp"):
        file_path = os.path.join(folder_path, file_name)
        gdfs.append(gpd.read_file(file_path))

# 모든 데이터프레임을 하나로 결합 (GeoDataFrame으로 변환)
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), geometry='geometry')

# 데이터 타입 확인 (디버깅용)
print(type(combined_gdf))  # <class 'geopandas.geodataframe.GeoDataFrame'> 이어야 함
print(combined_gdf.columns)  # 컬럼 확인
print(combined_gdf.head())  # 데이터 미리보기

# 동일한 EMD_CD 기준으로 Sur_Energy 평균 계산
if "EMD_CD" in combined_gdf.columns and "Sur_Energy" in combined_gdf.columns:
    averaged_gdf = combined_gdf.groupby("EMD_CD").agg({
        "Sur_Energy": "mean",  # Sur_Energy 평균 계산
        "geometry": "first"    # 첫 번째 geometry 유지
    }).reset_index()

    # 첫 번째 파일의 CRS(좌표 참조 시스템) 유지
    first_file = gdfs[0]
    averaged_gdf = gpd.GeoDataFrame(averaged_gdf, geometry='geometry')  # GeoDataFrame 변환
    averaged_gdf.set_crs(first_file.crs, inplace=True)  # CRS 설정

    # 결과 shp 파일 저장
    output_path = os.path.join(folder_path, "Average_summer.shp")
    averaged_gdf.to_file(output_path)
    print(f"결과가 {output_path}에 저장되었습니다.")
else:
    print("EMD_CD 또는 Sur_Energy 필드가 존재하지 않습니다.")

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['EMD_CD', 'COL_ADM_SE', 'EMD_NM', 'SGG_OID', 'NEAR_FID', 'NEAR_DIST',
       'NEAR_DIST3', 'pvgen', 'ElecUse', 'Sur_Energy', 'geometry'],
      dtype='object')
     EMD_CD COL_ADM_SE   EMD_NM  SGG_OID  NEAR_FID    NEAR_DIST   NEAR_DIST3  \
0  50110131      50110  50110̵3      145         1  5106.865277  5106.865277   
1  50110126      50110    50ȣ1̵      143         1  5900.005278  5900.005278   
2  50110123      50110    5ܵ11ϵ      142         1  8848.691253  8848.691253   
3  50110128      50110  50110̵2      141         1  3661.064454  3661.064454   
4  50110125      50110    50ȣ1ϵ      140         1  6051.042301  6051.042301   

   pvgen    ElecUse  Sur_Energy  \
0    0.0   4.753579   -3.327505   
1    0.0  13.086135   -9.160294   
2    0.0  36.072495  -25.250746   
3    0.0   2.507812   -1.755468   
4    0.0  17.486873  -12.240811   

                                            geometry  
0  POLYGON ((126.58464 33.51271, 126.584

In [32]:
#계절별 평균 계산_가을
import geopandas as gpd
import os
import pandas as pd

# 대상 폴더 경로
folder_path = r"F:\Junkyo\2025 Jeju Sector Coupling\average_fall70"

# 모든 shp 파일 읽고 결합
gdfs = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".shp"):
        file_path = os.path.join(folder_path, file_name)
        gdfs.append(gpd.read_file(file_path))

# 모든 데이터프레임을 하나로 결합 (GeoDataFrame으로 변환)
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), geometry='geometry')

# 데이터 타입 확인 (디버깅용)
print(type(combined_gdf))  # <class 'geopandas.geodataframe.GeoDataFrame'> 이어야 함
print(combined_gdf.columns)  # 컬럼 확인
print(combined_gdf.head())  # 데이터 미리보기

# 동일한 EMD_CD 기준으로 Sur_Energy 평균 계산
if "EMD_CD" in combined_gdf.columns and "Sur_Energy" in combined_gdf.columns:
    averaged_gdf = combined_gdf.groupby("EMD_CD").agg({
        "Sur_Energy": "mean",  # Sur_Energy 평균 계산
        "geometry": "first"    # 첫 번째 geometry 유지
    }).reset_index()

    # 첫 번째 파일의 CRS(좌표 참조 시스템) 유지
    first_file = gdfs[0]
    averaged_gdf = gpd.GeoDataFrame(averaged_gdf, geometry='geometry')  # GeoDataFrame 변환
    averaged_gdf.set_crs(first_file.crs, inplace=True)  # CRS 설정

    # 결과 shp 파일 저장
    output_path = os.path.join(folder_path, "Average_fall.shp")
    averaged_gdf.to_file(output_path)
    print(f"결과가 {output_path}에 저장되었습니다.")
else:
    print("EMD_CD 또는 Sur_Energy 필드가 존재하지 않습니다.")

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['EMD_CD', 'COL_ADM_SE', 'EMD_NM', 'SGG_OID', 'NEAR_FID', 'NEAR_DIST',
       'NEAR_DIST3', 'pvgen', 'ElecUse', 'Sur_Energy', 'geometry'],
      dtype='object')
     EMD_CD COL_ADM_SE   EMD_NM  SGG_OID  NEAR_FID    NEAR_DIST   NEAR_DIST3  \
0  50110131      50110  50110̵3      145         1  5106.865277  5106.865277   
1  50110126      50110    50ȣ1̵      143         1  5900.005278  5900.005278   
2  50110123      50110    5ܵ11ϵ      142         1  8848.691253  8848.691253   
3  50110128      50110  50110̵2      141         1  3661.064454  3661.064454   
4  50110125      50110    50ȣ1ϵ      140         1  6051.042301  6051.042301   

    pvgen    ElecUse  Sur_Energy  \
0  0.2860   8.222634   -5.469844   
1  0.1430  17.784361  -12.306053   
2  0.5885  56.411935  -38.899854   
3  0.4895   4.857447   -2.910713   
4  0.2035  25.454015  -17.614310   

                                            geometry  
0  POLYGON ((126.58464 33.51271, 1

In [33]:
#계절별 평균 계산_겨울
import geopandas as gpd
import os
import pandas as pd

# 대상 폴더 경로
folder_path = r"F:\Junkyo\2025 Jeju Sector Coupling\average_winter70"

# 모든 shp 파일 읽고 결합
gdfs = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".shp"):
        file_path = os.path.join(folder_path, file_name)
        gdfs.append(gpd.read_file(file_path))

# 모든 데이터프레임을 하나로 결합 (GeoDataFrame으로 변환)
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), geometry='geometry')

# 데이터 타입 확인 (디버깅용)
print(type(combined_gdf))  # <class 'geopandas.geodataframe.GeoDataFrame'> 이어야 함
print(combined_gdf.columns)  # 컬럼 확인
print(combined_gdf.head())  # 데이터 미리보기

# 동일한 EMD_CD 기준으로 Sur_Energy 평균 계산
if "EMD_CD" in combined_gdf.columns and "Sur_Energy" in combined_gdf.columns:
    averaged_gdf = combined_gdf.groupby("EMD_CD").agg({
        "Sur_Energy": "mean",  # Sur_Energy 평균 계산
        "geometry": "first"    # 첫 번째 geometry 유지
    }).reset_index()

    # 첫 번째 파일의 CRS(좌표 참조 시스템) 유지
    first_file = gdfs[0]
    averaged_gdf = gpd.GeoDataFrame(averaged_gdf, geometry='geometry')  # GeoDataFrame 변환
    averaged_gdf.set_crs(first_file.crs, inplace=True)  # CRS 설정

    # 결과 shp 파일 저장
    output_path = os.path.join(folder_path, "Average_winter.shp")
    averaged_gdf.to_file(output_path)
    print(f"결과가 {output_path}에 저장되었습니다.")
else:
    print("EMD_CD 또는 Sur_Energy 필드가 존재하지 않습니다.")

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['EMD_CD', 'COL_ADM_SE', 'EMD_NM', 'SGG_OID', 'NEAR_FID', 'NEAR_DIST',
       'NEAR_DIST3', 'pvgen', 'ElecUse', 'Sur_Energy', 'geometry'],
      dtype='object')
     EMD_CD COL_ADM_SE   EMD_NM  SGG_OID  NEAR_FID    NEAR_DIST   NEAR_DIST3  \
0  50110131      50110  50110̵3      145         1  5106.865277  5106.865277   
1  50110126      50110    50ȣ1̵      143         1  5900.005278  5900.005278   
2  50110123      50110    5ܵ11ϵ      142         1  8848.691253  8848.691253   
3  50110128      50110  50110̵2      141         1  3661.064454  3661.064454   
4  50110125      50110    50ȣ1ϵ      140         1  6051.042301  6051.042301   

    pvgen    ElecUse  Sur_Energy  \
0  0.7975   8.048721   -4.836605   
1  0.4675  24.525447  -16.700313   
2  1.3915  64.764266  -43.943486   
3  0.9680   4.736864   -2.347805   
4  0.6710  27.684154  -18.707908   

                                            geometry  
0  POLYGON ((126.58464 33.51271, 1

In [ ]:
#건물 발전량 계산

CSV 파일 읽기 에러 (fid=1): 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
CSV 파일 읽기 에러 (fid=2): 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
CSV 파일 읽기 에러 (fid=3): 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 14: invalid continuation byte

In [5]:
#건물 발전량 계산
import os
import pandas as pd
import geopandas as gpd

# 📌 CSV 인코딩 오류 방지 함수
def read_csv_with_fallback(file_path):
    """UTF-8로 CSV를 읽고, 실패하면 EUC-KR, CP949, Latin1 등의 인코딩을 순차적으로 시도"""
    encodings = ["utf-8", "euc-kr", "cp949", "latin1"]
    for enc in encodings:
        try:
            df = pd.read_csv(file_path, encoding=enc)
            print(f"✅ 파일 '{file_path}'을(를) {enc} 인코딩으로 성공적으로 읽음")
            return df
        except UnicodeDecodeError:
            print(f"⚠️ {enc} 인코딩으로 읽기 실패, 다음 인코딩 시도...")
    raise ValueError(f"❌ 모든 인코딩 시도 실패: {file_path}")

# 1. 파일 경로 설정
shp_path = r"E:\2025_Jeju_SectorCoupling\RooftopPVGen.shp"

csv_mapping = {
    1: r"C:\Users\82105\Desktop\Jeju_Database\2024_Jeju_Irradiance.csv",
    2: r"C:\Users\82105\Desktop\Jeju_Database\2024_Gosan_Irradiance.csv",
    3: r"C:\Users\82105\Desktop\Jeju_Database\2024_Seoguipo_Irradiance.csv"
}

# 2. 메인 shapefile 읽어오기 (필요한 필드는 'NEAR_FID'와 'A12'임)
gdf = gpd.read_file(shp_path, encoding='cp949')
for field in ["NEAR_FID", "A12"]:
    if field not in gdf.columns:
        raise ValueError(f"shapefile에 '{field}' 필드가 존재하지 않습니다.")

# 3. 각 NEAR_FID별로 처리: 해당하는 CSV 파일로부터 multiplier(계산 인자) 가져오기
#    CSV 파일은 첫 번째 행은 헤더, 두번째 행부터 데이터로 구성됨.
#    각 데이터 행에서, 첫번째 열은 출력 파일명, 두번째 열은 계산에 사용할 값.
for fid, csv_path in csv_mapping.items():
    # shapefile에서 해당 fid만 추출
    subset = gdf[gdf["NEAR_FID"] == fid].copy()
    if subset.empty:
        print(f"📝 NEAR_FID == {fid}인 피처가 없습니다. 다음 fid로 넘어갑니다.")
        continue

    if not os.path.exists(csv_path):
        print(f"CSV 파일이 존재하지 않음: {csv_path}")
        continue

    # CSV 파일을 인코딩 오류 방지 함수를 사용해 읽기
    df_csv = read_csv_with_fallback(csv_path)
    
    # CSV의 각 데이터(즉, 헤더 이후의 각 행)를 순회
    # 첫 번째 데이터 행(index 0)은 CSV의 2행, 그 이후는 3행부터 마지막행까지 처리
    for idx, row in df_csv.iterrows():
        # CSV의 첫번째 열: 출력 파일명으로 사용할 문자열 (공백 제거)
        out_filename = str(row.iloc[0]).strip()
        # CSV의 두번째 열: multiplier 값 (계산에 사용)
        try:
            multiplier = float(row.iloc[1])
        except Exception as e:
            print(f"⚠️ fid {fid}, CSV 행 {idx+2}의 multiplier값 변환 에러: {e}")
            continue

        # 계산: shapefile 속성의 A12 * 0.22 * multiplier
        # A12 값이 숫자임을 가정하며 계산 결과를 새 필드 'Calc_Value'에 저장
        subset["pvgen"] = subset["A12"].astype(float) * 0.22 * multiplier * 0.9

        # 출력 파일 경로 결정 (출력 폴더는 입력 shapefile과 같은 경로에서 생성)
        output_path = os.path.join(r"E:\2025_Jeju_SectorCoupling\BIPV", f"{out_filename}.shp")
        try:
            subset.to_file(output_path)
            print(f"✅ NEAR_FID {fid} - CSV 행 {idx+2} 처리 완료, 출력: {output_path}")
        except Exception as e:
            print(f"❌ 파일 저장 실패 ({output_path}): {e}")

print("모든 처리가 완료되었습니다.")

c:\Users\82105\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from cp949 to UTF-8.  This warning will not be emitted anymore
  return ogr_read(


⚠️ utf-8 인코딩으로 읽기 실패, 다음 인코딩 시도...
✅ 파일 'C:\Users\82105\Desktop\Jeju_Database\2024_Jeju_Irradiance.csv'을(를) euc-kr 인코딩으로 성공적으로 읽음


c:\Users\82105\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field A22 create as date field, though DateTime requested.
  ogr_write(
c:\Users\82105\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 419750101 of field A16 of feature 150961 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


KeyboardInterrupt: 